## *Análise de Sentimentos de Artigos*

Artigos Escolhidos

1970: Desníveis e falta de sinalização deixam via Anchieta mais perigosa

1970: Parlamento da Itália aprova a lei do divórcio; Vaticano reage

1970: Itália discute fazer referendo sobre divórcio após lei ser aprovada

1970: Programa de construção da ponte Rio-Niterói começa a ser revisto

1970: Missão da ONU diz que Portugal foi responsável por invasão da Guiné

1970: Inaugurado, viaduto sobre praça 14 Bis facilita ligação centro-zona sul

1970: Chanceler da Alemanha Ocidental chega à Polônia para assinar tratado

1970: Morre Abrahão de Moraes, um dos maiores astrônomos brasileiros

1970: Rio tem policiamento ostensivo após sequestro de embaixador suíço

1970: Governo aguarda por mensagem do embaixador sequestrado no Rio


In [ ]:
# Importações / Imports

# Web Scraping
from bs4 import BeautifulSoup 
import requests

# Mongo DB
from pymongo import MongoClient

# SpaCy Stopword WordCloud
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Análise de Sentimentos
# Sentiment Analysis
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

## *WEB SCRAPING*

In [ ]:
'''
Armazenando todas as URLs em uma lista para automatizar o processo em um loop
Collecting all URLs in a single list to automate the process in a loop
'''
urls = ['https://www1.folha.uol.com.br/banco-de-dados/2020/11/1970-desniveis-e-falta-de-sinalizacao-deixam-via-anchieta-mais-perigosa.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-parlamento-da-italia-aprova-a-lei-do-divorcio-vaticano-reage.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-italia-discute-fazer-referendo-sobre-divorcio-apos-lei-ser-aprovada.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-programa-de-construcao-da-ponte-rio-niteroi-comeca-a-ser-revisto.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-missao-da-onu-diz-que-portugal-foi-responsavel-por-invasao-da-guine.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-inaugurado-viaduto-sobre-praca-14-bis-facilita-ligacao-centro-zona-sul.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-chanceler-da-alemanha-ocidental-chega-a-polonia-para-assinar-tratado.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-morre-abrahao-de-moraes-um-dos-maiores-astronomos-brasileiros.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-rio-tem-policiamento-ostensivo-apos-sequestro-de-embaixador-suico.shtml',
        'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-governo-aguarda-por-mensagem-do-embaixador-sequestrado-no-rio.shtml']

In [ ]:
'''
O loop tem a função de juntar todos os paragráfos de cada artigo como um elemento só e agregar todos eles em uma lista, desta forma, temos todos os 10 textos e cada parágrafo em uma lista específica para realizar a análise de sentimentos posteriormente.
The loop has the function of join all paragraphs of each article in a single element and aggregate all of them in a list, this way, we have all texts and each paragraph in a specific list to do the sentiment analysis after.
'''
textos = []
paragrafos = []
for iterador in range(len(urls)):
    r = requests.get(urls[iterador])
    soup = BeautifulSoup(r.content, 'html.parser')
    texto = ''
    p_artigo = soup.find('div', class_='c-news__body').findAll('p')
    for p in p_artigo:
        paragrafos.append(p.text)
        texto += ''.join(p.findAll(text = True))
    textos.append(texto)

In [ ]:
'''
Conferindo se ocorreu como esperava.
Checking if occured as expected.
'''
print(textos)
print(f'Tamanho da lista: {len(textos)}')

## *MONGO DB*

In [ ]:
'''
Criando lista com o nome dos artigos para gravar como o nome no banco de dados e o valor sendo o texto do nome correspondente ao artigo.
Salvando todos eles no MongoDB com o nome da coleção de 'textos'.
Creating a list with the articles's name to store in the database e the value being the text of the corresponding article.
Saving all of them in the Mongo DB with the collection's name 'textos'.
'''
artigos = ["Desníveis e falta de sinalização deixam via Anchieta mais perigosa", "Parlamento da Itália aprova a lei do divórcio; Vaticano reage", "Itália discute fazer referendo sobre divórcio após lei ser aprovada", "Programa de construção da ponte Rio-Niterói começa a ser revisto",
"Missão da ONU diz que Portugal foi responsável por invasão da Guiné", "Inaugurado, viaduto sobre praça 14 Bis facilita ligação centro-zona sul",
"Chanceler da Alemanha Ocidental chega à Polônia para assinar tratado", "Morre Abrahão de Moraes, um dos maiores astrônomos brasileiros",
"Rio tem policiamento ostensivo após sequestro de embaixador suíço", "Governo aguarda por mensagem do embaixador sequestrado no Rio"]

dict_textos = dict()

for i in range(len(textos)):
    dict_textos[artigos[i]] = textos[i]

client = MongoClient()
name = 'textos'
db = client[name]
collection = db[name]
collection.insert_one(dict_textos)


## *STOPWORDS, NUVEM DE PALAVRAS, VERBOS E ENTIDADES*

In [ ]:
'''
Gerando WordCloud para cada texto.
Transforma cada item da lista 'textos' em uma string, passa para o doc, remove as stopwords e cria o Word Cloud.
Generating WordCloud for each text.
Transform each list item into a string, moves to doc, remove the stopwords e create the Word Cloud.
'''
nlp = spacy.load("pt_core_news_sm")
for i in range(len(textos)):
    texto = str(textos[i])
    doc = nlp(texto)
    tokens = [token for token in doc if not token.is_stop and token.is_punct != True]
    str_tokens = ' '.join([str(item) for item in tokens])
    wordcloud = WordCloud(width=800, height=600, background_color='black', min_font_size=10).generate(str_tokens)
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.show()

In [ ]:
'''
Coletando os verbos de todos textos, criando um dicionário relacionando o artigo com seus respectivos verbos e inserindo-os no Mongo DB.
Collecting the verbs of all texts, creating a dictionary relating the article with its respective verbs and storing in Mongo DB.
'''
verbos_list = []
for i in range(len(textos)):
    nlp = spacy.load("pt_core_news_sm")
    texto = str(textos[i])
    doc = nlp(texto)
    verbos = [token.text for token in doc if token.pos_ == "VERB"]
    verbos_list.append(verbos)

In [ ]:
dict_verbos = dict()

for i in range(len(textos)):
    dict_verbos[artigos[i]] = verbos_list[i][:]

collection.insert_one(dict_verbos)

In [ ]:
'''
Coletando as entidades de todos textos, criando um dicionário relacionando o artigo com suas respectivas entidades e inserindo-as no Mongo DB.
Collecting the entities of all texts, creating a dictionary relating the article with its respective entities and storing in Mongo DB.
'''
entidades_list = []
for i in range(len(textos)):
    nlp = spacy.load("pt_core_news_sm")
    texto = str(textos[i])
    doc = nlp(texto)
    entidades = doc.ents
    entidades_list.append(str(entidades))

In [ ]:
dict_entidades = dict()

for i in range(len(textos)):
    dict_entidades[artigos[i]] = entidades_list[i][:]

collection.insert_one(dict_entidades)

## *ANÁLISE DE SENTIMENTOS*

In [ ]:
'''
Carregando o arquivo CSV.
Loading CSV File
'''
df = pd.read_csv('Tweets_Mg.csv')

In [ ]:
'''
Selecionando dados. Os únicos relevantes são o texto e a classificação.
Selecting data. The only relevant are text and classification.
'''
texto_tt = df.Text.values
classificacao = df.Classificacao.values

'''
Treinando o modelo baseado nos tweets.
Training the model based on tweets.
'''
vector = CountVectorizer()
f_twt = vector.fit_transform(texto_tt)

model = MultinomialNB()
model.fit(f_twt, classificacao)

'''
Testando o modelo.
Testing the model.
'''
teste_textos = vector.transform(textos)
preditos_textos = model.predict(teste_textos) # Todos deram como resultado 'neutro'.

teste_paragrafos = vector.transform(paragrafos)
preditos_paragrafos = model.predict(teste_paragrafos)

In [ ]:
'''
Avaliando o modelo.
Evaluating the model.
'''
resultados = cross_val_predict(model, f_twt, classificacao, cv = 10)
score = metrics.accuracy_score(classificacao, resultados)

In [ ]:
'''
Inserindo os valores preditos pelo modelo e seu score no Mongo DB.
Storing the predicted values by model and its score in Mongo DB.
'''
dict_preditos_score_textos = dict()
dict_preditos_score_textos['Preditos'] = list(preditos_textos)
dict_preditos_score_textos['Score'] = score

dict_preditos_paragrafos = dict()
dict_preditos_paragrafos['Preditos Paragrafos'] = list(preditos_paragrafos)

collection.insert_one(dict_preditos_score_textos)
collection.insert_one(dict_preditos_paragrafos)

print(pd.crosstab(classificacao, resultados, rownames = ["Real"], colnames=["Predito"], margins=True))

Autor: Victor Balbino Araujo | Avaliação Compasso